Samler data for temperatur, lufttrykk og nedbør i en csv fil:

In [2]:
import pandas as pd

# Leser inn de 3 filene for vær verdier
df_temp = pd.read_csv("../data/R_Temperatur (°C).csv")
df_trykk = pd.read_csv("../data/R_Lufttrykk (hPa).csv")
df_nedbør = pd.read_csv("../data/R_Nedbør (mm).csv")

# Slår sammen filene til en fil, basert på dato
samlet = df_temp \
    .merge(df_trykk, on="Dato", how="left") \
    .merge(df_nedbør, on="Dato", how="left")


# Oppretter ny fil for samlet verdier
samlet.to_csv("R_Sammenslaaing_gjsnitt.csv", index=False)

print(samlet.head())


         Dato  Temperatur (°C)  Lufttrykk (hPa)  Nedbør (mm)
0  2023-01-01              0.5           1004.1          4.5
1  2023-01-02             -5.4           1011.0          0.0
2  2023-01-03             -6.4           1017.9          0.1
3  2023-01-04             -6.2           1011.4          0.0
4  2023-01-05             -3.1           1012.7          0.0


Identifisere og håndtere manglende verdier i datasettet:

In [5]:
samlet.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Dato                          183 non-null    object 
 1   Temperatur (°C) daglig snitt  183 non-null    float64
 2   Lufttrykk (hPa)               183 non-null    float64
 3   Nedbør (mm)                   183 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.1+ KB


In [6]:
print(samlet.isna().sum())

Dato                            0
Temperatur (°C) daglig snitt    0
Lufttrykk (hPa)                 0
Nedbør (mm)                     0
dtype: int64


Kode for å lokalisere "NaN" verdier:

In [4]:
import pandas as pd

df_sammenslaaing = pd.read_csv("../data/R_Sammenslaaing_gjsnitt.csv")

df_sammenslaaing.loc[df_sammenslaaing.isnull().any(axis=1)]


,Dato,Temperatur (°C),Lufttrykk (hPa),Nedbør (mm)
5,2023-01-06,-3.0,NaN,0.0
33,2023-02-03,NaN,1016.8,5.7


Lager ny csv for oppdaterte (riktige) verdier:

In [9]:
df_R_Sammenslaaing_UtenFeil = df_sammenslaaing.copy()

Kode for å rette opp i eventuelle negative nedbør og/eller lufttrykk-verdier:

In [12]:
df_R_Sammenslaaing_UtenFeil['Nedbør (mm)'] = [abs(x) for x in df_R_Sammenslaaing_UtenFeil['Nedbør (mm)']]
df_R_Sammenslaaing_UtenFeil.to_csv("../data/R_Sammenslaaing_uten_feil.csv")

df_R_Sammenslaaing_UtenFeil['Lufttrykk (hPa)'] = [abs(x) for x in df_R_Sammenslaaing_UtenFeil['Lufttrykk (hPa)']]
df_R_Sammenslaaing_UtenFeil.to_csv("../data/R_Sammenslaaing_uten_feil.csv")

Gjør at datoene blir i datetime-format så ikke det er noen skrivefeil i datoene henta:

In [13]:
# Gjør at 'Dato' er i datetime-format
df_R_Sammenslaaing_UtenFeil['Dato'] = pd.to_datetime(df_R_Sammenslaaing_UtenFeil['Dato'])

# Start fra første dato og generer resten
start_dato = df_R_Sammenslaaing_UtenFeil.loc[0, 'Dato']
riktige_datoer = [start_dato + pd.Timedelta(days=i) for i in range(len(df_R_Sammenslaaing_UtenFeil))]

# Sett inn de riktige datoene
df_R_Sammenslaaing_UtenFeil['Dato'] = riktige_datoer


print(df_R_Sammenslaaing_UtenFeil.head())

        Dato  Temperatur (°C)  Lufttrykk (hPa)  Nedbør (mm)
0 2023-01-01              0.5           1004.1          4.5
1 2023-01-02             -5.4           1011.0          0.0
2 2023-01-03             -6.4           1017.9          0.1
3 2023-01-04             -6.2           1011.4          0.0
4 2023-01-05             -3.1           1012.7          0.0


Kode for å rette opp i ussansynlige temperaturverdier, samt itterere for å sjekke om en temperatur ikke gir verdier(NaN). Bytter ut disse mistenkelige verdiene med ett gjennomsnitt fra dagen før opg dagen etter.

In [15]:
import pandas as pd

# Les inn CSV-filen
df = pd.read_csv("../data/R_Sammenslaaing_gjsnitt.csv")

# Gå gjennom temperaturkolonnen med indekser
for i in range(1, len(df) - 1):  
    temp = df.loc[i, "Temperatur (°C)"]

    # Hvis verdien er mistenkelig eller tom
    if pd.isna(temp) or temp <= -50 or temp >= 50:

        # Gjennomsnitt av dagen før og dagen etter
        temp_forrige = df.loc[i - 1, "Temperatur (°C)"]
        temp_etter = df.loc[i + 1, "Temperatur (°C)"]
        gjennomsnitt = (temp_forrige + temp_etter) / 2

        # Erstatt verdien
        df.loc[i, "Temperatur (°C)"] = gjennomsnitt

# Skriv til filen uten feil
df.to_csv("../data/R_Sammenslaaing_uten_feil.csv", index=False)

Kode for å teste om lufttrykk-verdien er tom:

In [17]:
import pandas as pd

# Les inn CSV-filen
df = pd.read_csv("../data/R_Sammenslaaing_gjsnitt.csv")

# Gå gjennom temperaturkolonnen med indekser
for i in range(1, len(df) - 1):  
    trykk = df.loc[i, "Lufttrykk (hPa)"]

    # Hvis verdien er mistenkelig eller tom
    if pd.isna(trykk):

        # Gjennomsnitt av dagen før og dagen etter
        trykk_forrige = df.loc[i - 1, "Lufttrykk (hPa)"]
        trykk_etter = df.loc[i + 1, "Lufttrykk (hPa)"]
        gjennomsnitt = (trykk_forrige + trykk_etter) / 2

        # Erstatt verdien
        df.loc[i, "Lufttrykk (hPa)"] = gjennomsnitt

# Skriv til filen uten feil
df.to_csv("../data/R_Sammenslaaing_uten_feil.csv", index=False)